# Visualization for Sakila Movie Database

This notebook is used for generating visualizations based on the PostgreSQL data analysis results.

Several interesting questions have been explored as we can see below.

**Table of Contents**

- [Query 1. What is the rental behavior in family movies?](#query1)

- [Query 2. What were the rental duration in quartile for family movies?](#query2)

- [Query 3. What were the rental orders by store?](#query4)

- [Query 4. Payment Difference between months for Top 10 Payors](#query6)




In [1]:
# change to working directory
import os
os.chdir('projects_on_GitHub/udacity_project/SQL/Sakila_Query_Viz')

In [2]:
ls

README.md
Visualization_for_Sakila_QueryResult.ipynb
query.sql
query1.csv
query2.csv
query3.csv
query4.csv
query5.csv
query6.csv


In [3]:
df1 = pd.read_csv('query1.csv')
df1.head()

,film_title,category_name,rental_count
0,Alter Victory,Animation,22
1,Anaconda Confessions,Animation,21
2,Bikini Borrowers,Animation,17
3,Blackout Private,Animation,27
4,Borrowers Bedazzled,Animation,22


In [4]:
# construct dataframe for plot bar graph
df1 = df1.groupby('category_name').rental_count.sum().sort_values(ascending=False).to_frame().reset_index()
df1

,category_name,rental_count
0,Animation,1166
1,Family,1096
2,Children,945
3,Comedy,941
4,Classics,939
5,Music,830


In [ ]:
# Configure some 
figsize=(8, 4)
dpi = 100
label_fontsize = 12
title_fontsize = 18
title_pad = 15
label_pad = 6